# Port Statistics

This notebook develops and explores the various port statistics used in the [Port Performance Project](https://github.com/epistemetrica/Port-Performance-Project). See the README.md file in the main directory for more info.

The primary data set comes from a combination of AIS vessel data and port data, processed in the Port Geodata notebook.

Statistics and final dataframes developed here are used in the Port Performance Dashboard.



In [2]:
#prelims
import polars as pl
import polars.selectors as cs
import pandas as pd
import geopandas as gpd
import time
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import glob
import folium
from folium.plugins import HeatMap

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=100);

## Load Data from geodata_prep notebook

In [3]:
#load and inspect
lf = pl.scan_parquet('port data/dashboard/main.parquet')
display(lf.describe())
lf.limit(5).collect()

statistic,docking_id,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,status_duration,dock_id,dock_name,facility_type,port_name,dist_to_dock,year,month,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon
str,f64,str,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,str,f64,f64,str,f64,f64,f64,f64,f64,f64
"""count""",2.300198e6,"""2300198""","""2300198""",2.300198e6,2.291155e6,2.24229e6,2.300198e6,"""2300198""",2.300198e6,2.300182e6,2.300198e6,2.145356e6,2.105906e6,1.810074e6,"""2164665""",2.299448e6,"""2300198""","""2300198""","""2290526""","""2300198""",2.300198e6,2.300198e6,"""2300198""",2.300198e6,2.300198e6,2.300198e6,2.300198e6,2.300198e6,2.300198e6
"""null_count""",0.0,"""0""","""0""",0.0,9043.0,57908.0,0.0,"""0""",0.0,16.0,0.0,154842.0,194292.0,490124.0,"""135533""",750.0,"""0""","""0""","""9672""","""0""",0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",3.9815e6,null,"""2021-04-17 10:11:48.299924""",0.381967,185.715425,183.363903,2.640737,null,72.653523,9.9809e6,210.445869,32.067916,10.949487,73.238112,null,2903.40313,null,null,null,null,72578.516239,2020.812705,null,33.022036,-98.445398,32.993502,-98.504472,32.991448,-98.505129
"""std""",2.3018e6,null,null,3.7607,90.002125,99.69196,2.490101,null,4.564537,2.3781e7,60.995697,7.93468,2.971257,8.076278,null,37808.128169,null,null,null,null,554382.347274,1.896685,null,6.186134,20.294724,6.149815,20.106922,6.149675,20.105812
"""min""",139.0,"""205042000""","""2018-01-01 00:11:06""",0.0,0.0,0.0,0.0,null,70.0,0.0,101.0,0.0,-12.8,0.0,"""Albany Port District, NY""",0.0,"""00XE""","""ADM Corpus Christi Grain Eleva…","""Anchorage""","""Albany Port District, NY""",0.117339,2018.0,"""201801""",5.47647,-176.77362,17.936081,-166.53444,17.938939,-166.549916
"""25%""",1.890288e6,null,"""2019-08-21 11:34:09""",0.0,127.0,92.0,0.0,null,70.0,9.311311e6,179.0,28.0,8.8,70.0,null,32.0,null,null,null,null,81.882034,2019.0,null,29.73062,-118.24647,29.73497,-118.249201,29.747023,-118.268099
"""50%""",3.986812e6,null,"""2021-05-12 23:37:49""",0.0,186.6,180.0,5.0,null,70.0,9.484405e6,189.0,32.0,10.9,70.0,null,75.0,null,null,null,null,139.328521,2021.0,null,32.09011,-94.09073,32.089076,-94.091438,32.084711,-94.094985
"""75%""",6.001762e6,null,"""2022-10-22 17:05:24""",0.1,252.1,269.0,5.0,null,74.0,9.668269e6,229.0,32.0,13.1,79.0,null,186.0,null,null,null,null,256.70334,2022.0,null,37.79395,-80.6067,37.794167,-80.605556,37.806665,-80.60815
"""max""",7.99451e6,"""725019920""","""2024-09-30 23:53:26""",102.3,406.0,365.0,15.0,null,89.0,9.849033e8,667.0,86.0,25.5,217.0,"""Yabucoa, PR""",3.469009e6,"""1JHK""","""YUSEN TERMINALS BERTHS 212-221""","""Tie Off""","""Yabucoa, PR""",3.3858e7,2024.0,"""202409""",80.43748,175.05822,61.24306,-65.834952,61.23778,-65.836226


docking_id,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,status_duration,dock_id,dock_name,facility_type,port_name,dist_to_dock,year,month,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon
u32,str,datetime[μs],f64,f64,f64,f64,cat,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,str,f64,i32,str,f64,f64,f64,f64,f64,f64
139,"""205042000""",2022-12-12 13:14:21,0.2,289.6,319.0,1.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,null,45.0,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",25920.228532,2022,"""202212""",33.62545,-118.04819,33.757222,-118.21888,33.73957,-118.2095
139,"""205042000""",2022-12-11 15:05:03,1.1,93.9,172.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,null,81.0,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",25194.50927,2022,"""202212""",33.6304,-118.05158,33.757222,-118.21888,33.73957,-118.2095
139,"""205042000""",2022-12-11 16:26:07,0.4,315.8,214.0,1.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,null,786.0,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",25497.836741,2022,"""202212""",33.62964,-118.04873,33.757222,-118.21888,33.73957,-118.2095
139,"""205042000""",2022-12-12 05:32:10,1.3,243.9,256.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,null,45.0,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",25846.119841,2022,"""202212""",33.62734,-118.04701,33.757222,-118.21888,33.73957,-118.2095
139,"""205042000""",2022-12-12 06:17:11,0.0,255.1,255.0,1.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,null,225.0,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",25826.354092,2022,"""202212""",33.62756,-118.04701,33.757222,-118.21888,33.73957,-118.2095


## Call ID vs Docking ID

In [ ]:
#count simultaneous port waters
df = (
    lf.select('which_port_waters').collect()
    .with_columns(
        port_waters1 = (
            pl.col('which_port_waters').str.split('|').list.get(0, null_on_oob=True)
        ),
        port_waters2 = (
            pl.col('which_port_waters').str.split('|').list.get(1, null_on_oob=True)
        ),
    )
)

df.describe()

statistic,which_port_waters,port_waters1,port_waters2
str,str,str,str
"""count""","""2164665""","""2164665""","""495838"""
"""null_count""","""135533""","""135533""","""1804360"""
"""mean""",null,null,null
"""std""",null,null,null
"""min""","""Albany Port District, NY""","""Albany Port District, NY""","""Port of Longview, WA"""
"""25%""",null,null,null
"""50%""",null,null,null
"""75%""",null,null,null
"""max""","""Yabucoa, PR""","""Yabucoa, PR""","""Texas City, TX"""


In [20]:
df

which_port_waters,count
str,u32
"""Greater Lafourche Port, LA""",2376
"""Nawiliwili, Kaua'i, HI""",593
"""Port of Savannah, GA""",43449
"""Kahului, Maui, HI""",1151
"""Texas City, TX""",50195
"""New Orleans, LA""",68749
"""Stockton, CA""",46179
"""Port of Kalama, WA""",18346
"""San Diego Unified Port, CA""",22982


In [ ]:
#collapse docking ids from the same dock in the same day
lf1 = (
    lf
    .with_columns(
        pl.when(
            #the status is docked and
            (pl.col('status')==5) &
            #the same vessel is associated with the same dock and
            (pl.col('dock_id')==pl.col('dock_id').shift()).over('mmsi') &
            #the docking_id changes and
            (pl.col('docking_id')!=pl.col('docking_id').shift()).over('mmsi') &
            #the time difference is less than 1 day
            ((pl.col('time')-pl.col('time').shift())<86400)
        )
        #then set the docking_id to the previous docking_id
        .then(pl.col('docking_id').shift())
        #otherwise keep the original docking_id
        .otherwise(pl.col('docking_id')).alias('docking_id')
    )
    #unify docking ids that occur between same docking ids
    #ensure sorting
    .sort('mmsi','time')
)

lf1.limit(5).collect()

In [18]:
id = pl.DataFrame({"docking_id": [1, 2, 2, 2, 2, 3, 3, 2, 2]})
id

docking_id
i64
1
2
2
2
3
3
2
2
4


In [14]:
df = lf1.select('mmsi', 'time', 'docking_id').collect()
df.head()

mmsi,time,docking_id
str,datetime[μs],u32
"""205042000""",2022-12-11 12:23:06,139
"""205042000""",2022-12-11 15:05:03,139
"""205042000""",2022-12-11 16:26:07,139
"""205042000""",2022-12-12 05:32:10,139
"""205042000""",2022-12-12 06:17:11,139


In [ ]:
smalldf = (
    df
    #ensure sorting
    .sort('mmsi', 'time')
    #keep only changes in docking_id
    .filter(df['docking_id']!=df['docking_id'].shift())
    #create indicator for whether id has been seen before
    .with_columns(
        pl.when(pl.col('docking_id').is_in)
    )
)

smalldf.head()

mmsi,time,docking_id
str,datetime[μs],u32
"""205042000""",2022-12-26 00:24:27,147
"""205042000""",2022-12-26 04:06:27,149
"""205087000""",2021-04-08 19:29:52,335
"""205087000""",2021-04-12 20:45:31,343
"""205087000""",2021-04-24 00:00:34,345


## Generate Stats

In [3]:
#define calls lazyframe for building stats
calls_lf = (
    lf
    #grouby by port dock and docking id
    .group_by(['port_name', 'dock_id', 'docking_id'])
    .agg(
        #duration of docking_id
        docking_id_duration = (
            pl.col('time').max() - pl.col('time').min()
        ).dt.total_minutes(),
        #vessel mmsi
        mmsi = pl.first('mmsi'),
        #vessel size
        vessel_size = pl.first('length'),
        #time of mooring
        docking_time = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
            .drop_nulls() #polars .first() evals to null if first val is null
        ).first(),
        #time a berth
        hrs_at_berth = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum()/60
        ),
        #time at anchor
        hrs_at_anchor = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum()/60
        ),
        #time entering port waters - not working as expected
        time_port_entry =(
            pl.when(pl.col('in_port_waters')==True) #need related port, not any ports
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
            .min()),
        #time in port waters to berth
        hrs_in_port_waters = (
            (pl.when(pl.col('status')==5)
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
            .min()) - 
            (pl.when(pl.col('in_port_waters')==True)
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
            .min())
        ).dt.total_hours(),
        #inspect in port waters
        sum_in_port_waters = pl.col('in_port_waters').sum()
    )
)

In [4]:
calls_lf.limit(5).collect()

port_name,dock_id,docking_id,docking_id_duration,mmsi,vessel_size,docking_time,hrs_at_berth,hrs_at_anchor,time_port_entry,hrs_in_port_waters,sum_in_port_waters
str,str,u32,i64,str,f64,datetime[μs],f64,f64,datetime[μs],i64,u32
"""Seattle, WA""","""016P""",8622653,21,"""636092866""",262.0,2022-08-17 07:05:03,1.7,0.0,2022-08-17 06:44:02,0,2
"""San Diego Unified Port, CA""","""0V92""",1966606,27,"""311000415""",189.0,2019-01-02 01:32:53,4.4,0.0,2019-01-02 01:05:53,0,2
"""Port of Los Angeles, CA""","""0URN""",6433845,174,"""538007480""",340.0,2020-10-26 01:51:58,0.733333,0.0,2020-10-25 22:57:58,2,2
"""Baltimore, MD""","""0YRT""",1691154,84,"""304617000""",137.0,2021-06-08 09:33:25,3.133333,0.0,2021-06-08 08:09:21,1,2
"""Baltimore, MD""","""0YZQ""",2108952,6,"""311044300""",199.0,2019-07-30 20:28:33,0.3,0.0,2019-07-30 20:22:33,0,2


In [5]:
calls_lf.select('docking_id_duration').describe()

statistic,docking_id_duration
str,f64
"""count""",1.376181e6
"""null_count""",0.0
"""mean""",4534.083848
"""std""",48425.784125
"""min""",0.0
"""25%""",9.0
"""50%""",33.0
"""75%""",83.0
"""max""",3.285404e6


In [6]:
calls_lf.select('docking_id_duration').with_columns(pl.all().replace(0,None)).describe()

statistic,docking_id_duration
str,f64
"""count""",1.148967e6
"""null_count""",227214.0
"""mean""",5430.721721
"""std""",52952.188826
"""min""",1.0
"""25%""",18.0
"""50%""",45.0
"""75%""",98.0
"""max""",3.285404e6


In [7]:
calls_lf.select('sum_in_port_waters').collect().to_series().value_counts().sort(by='count')

sum_in_port_waters,count
u32,u32
91,1
26,1
49,1
33,1
58,1
50,1
83,1
28,1
54,1


In [ ]:
#create monthly stats dataframe
monththly_df = (
    calls_lf
    #get month from docking time
    .with_columns(
        #extract month from docking time
        month = pl.col('docking_time').dt.strftime('%Y%m')
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'month'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('docking_id'),
        #time at dock stats for each vessel in hours
        hrs_occupied = pl.sum('hrs_at_berth'),
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor'),
        #time in port waters 
        hrs_in_port_waters_total = pl.sum('hrs_in_port_waters'),
        hrs_in_port_waters_mean = pl.mean('hrs_in_port_waters'),
        hrs_in_port_waters_median = pl.median('hrs_in_port_waters')
    )
    #get hours from each month
    .with_columns(
        hrs_in_month = (
            pl.when(pl.col('month').str.tail(2).is_in(['01', '03', '05', '07', '08', '10', '12']))
            .then(31*24)
            .when(pl.col('month').str.tail(2).is_in(['04', '06', '09', '11']))
            .then(30*24)
            .otherwise(28*24)
        )
    )
    .with_columns(
        #dock utilization - percentage of time a dock is occupied
        utilization = (
            pl.col('hrs_occupied')/pl.col('hrs_in_month')
        )
    )
    #drop hours in month
    .drop('hrs_in_month')
    #sort by port dock then month
    .sort(['port_name', 'dock_id', 'month'])
    #collect
    .collect()
)

#inspect
monththly_df.head()

In [ ]:
monththly_df.with_columns(pl.all().replace(0,None)).describe()

In [ ]:
#create annual port stats dataframe
annual_df = (
    calls_lf
    #get year from docking time
    .with_columns(
        #extract year from docking time
        year = pl.col('docking_time').dt.year()
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'year'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('docking_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth')/60,
        hrs_at_berth_mean = pl.mean('hrs_at_berth')/60,
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor')/60,
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')/60
    )
    #sort by port then month
    .sort(['port_name', 'year'])
    #collect
    .collect()
)

#inspect
annual_df.head()

In [ ]:
#get point in time stats

#create point in time (pit) df to join stats to
pit_df = (
    lf
    .with_columns(date = pl.col('time').dt.date())
    .select('port_name', 'dock_id', 'month', 'date')
    .unique().collect()
)

for hour in range(0, 24):
    #create a time object for each hour
    hour_dt = pl.time(hour)
    #create a dataframe for each hour
    hour_df = (
        lf
        .with_columns(
            #get end of status time
            end_time = (pl.col('time') + 
                        pl.duration(minutes=pl.col('status_duration'))),
            #get date from time
            date = pl.col('time').dt.date()
        )
        #group by port dock and hour
        .group_by(['port_name', 'dock_id', 'month', 'date'])
        .agg(
            #number of vessels at dock at each hour
            vessels_at_dock = (
                #when moored at hour
                pl.when((pl.col('status')==5) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                         pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls() #n_unique counts nulls as unique values
                .n_unique()
            ),
            #number of vessels at anchor at each hour
            vessels_at_anchor = (
                #when anchored at hour
                pl.when((pl.col('status')==1) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                        pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls()
                .n_unique()
            )
        )
        .collect()
    )
    #join the hour dataframe to the main dataframe
    pit_df = (
        pit_df
        .join(hour_df, 
              on=['port_name', 'dock_id', 'month', 'date'], 
              how='left')
        #rename the columns to include the hour
        .rename({
            'vessels_at_dock': f'vessels_at_dock_{hour}',
            'vessels_at_anchor': f'vessels_at_anchor_{hour}'
        })
    )

#get port stats by month
pit_df = (
    pit_df
    #group by port and date
    .group_by(['port_name', 'month', 'date'])
    .agg(
        #sum the number of vessels at dock at each hour
        cs.starts_with('vessels_at_dock_').sum(),
        #sum the number of vessels at anchor at each hour
        cs.starts_with('vessels_at_anchor_').sum()
    )
    #get the max at any hour
    .with_columns(
        #get max at dock at any hour
        vessels_at_dock_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get mean at dock any hour
        vessels_at_dock_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get max at anchor at any hour
        vessels_at_anchor_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_anchor_'))
        ),
        #get mean at anchor any hour
        vessels_at_anchor_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_anchor_'))
        )
    )
    #select the columns to keep
    .select(['port_name', 'month', 'date', 'vessels_at_dock_max', 
             'vessels_at_dock_mean', 'vessels_at_anchor_max',
             'vessels_at_anchor_mean'])
    #aggregate by month
    .group_by(['port_name', 'month'])
    .agg(
        #get max at anchor on any date
        vessels_at_anchor_max = pl.max('vessels_at_anchor_max'),
        #get mean at anchor on any date
        vessels_at_anchor_mean = pl.mean('vessels_at_anchor_mean'),
        #get max at dock on any date
        vessels_at_dock_max = pl.max('vessels_at_dock_max'),
        #get mean at dock on any date
        vessels_at_dock_mean = pl.mean('vessels_at_dock_mean'
        )
    )
)

In [ ]:
#inspect
pit_df.head()

In [ ]:



    
    #group by month to get averages
    .group_by(['port_name', 'month'])
    .agg(
        #average number of vessels at dock
        vessels_at_dock_at_noon_mean = pl.mean('vessels_at_dock'),
        #average number of vessels at anchor
        vessels_at_anchor_at_noon_mean = pl.mean('vessels_at_anchor')
    )
    #sort by port then date
    .sort(['port_name', 'month'])
    #collect
    .collect()
)

#inspect
df.head()

#join to stats dfs



In [ ]:
#get means for each port over all time
port_alltime_df = (
    port_monththly_df
    .group_by(['port_name'])
    .agg(
        #average number of vessels at dock
        vessels_at_dock_at_noon_mean = pl.mean('vessels_at_dock_at_noon_mean'),
        #average number of vessels at anchor
        vessels_at_anchor_at_noon_mean = pl.mean('vessels_at_anchor_at_noon_mean'),
        #average time at dock in hours
        hrs_at_berth_median = pl.mean('hrs_at_berth_median'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth_mean'),
        #average time at anchor in hours
        hrs_at_anchor_median = pl.mean('hrs_at_anchor_median'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor_mean')
    )
    #sort by port
    .sort(['port_name'])
)

#inspect
port_alltime_df.head()